In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# 准备数据
train_data = ... # 成对的输入序列和目标序列
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# 定义编码器
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded)
        return hidden

# 定义解码器
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = nn.functional.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

# 定义Seq2Seq模型
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_tensor, target_tensor):
        encoder_hidden = self.encoder(input_tensor)
        decoder_input = torch.tensor([[SOS_token]]) # SOS_token是起始符号的索引
        decoder_hidden = encoder_hidden

        outputs = []
        for di in range(target_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs.append(decoder_output)
            decoder_input = target_tensor[di] # Teacher forcing

        return torch.stack(outputs)

# 初始化模型
input_size = ... # 输入词汇表的大小
output_size = ... # 输出词汇表的大小
hidden_size = 256
encoder = Encoder(input_size, hidden_size)
decoder = Decoder(hidden_size, output_size)
model = Seq2Seq(encoder, decoder)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    for i, (input_tensor, target_tensor) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(input_tensor

In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)

    def forward(self, input):
        outputs, (hidden, cell) = self.lstm(input)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size):
        super().__init__()
        self.lstm = nn.LSTM(hidden_size, output_size)
        self.linear = nn.Linear(output_size, output_size)

    def forward(self, input, hidden):
        outputs, (hidden, cell) = self.lstm(input, hidden)
        outputs = self.linear(outputs)
        return outputs, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.encoder = Encoder(input_size, hidden_size)
        self.decoder = Decoder(output_size, hidden_size)

    def forward(self, input, target):
        encoder_hidden, encoder_cell = self.encoder(input)
        decoder_hidden, decoder_cell = self.decoder(target[:, 0], encoder_hidden)
        for i in range(1, target.size(1)):
            decoder_output, decoder_hidden, decoder_cell = self.decoder(target[:, i], decoder_hidden)
        output = decoder_output
        return output

def train(model, train_data, val_data, epochs):
    for epoch in range(epochs):
        model.train()
        for input, target in train_data:
            output = model(input, target)
            loss = nn.CrossEntropyLoss()(output, target.view(-1))
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for input, target in val_data:
                output = model(input, target)
                _, predicted = output.max(1)
                correct += (predicted == target).sum().item()
                total += target.size(0)
            val_accuracy = correct / total
        print("Epoch {}: train loss {:.4f}, val accuracy {:.4f}".format(epoch + 1, loss.item(), val_accuracy))

if __name__ == "__main__":
    input_size = 100
    hidden_size = 50
    output_size = 100

    train_data = torch.load("train_data.pt")
    val_data = torch.load("val_data.pt")

    model = Seq2Seq(input_size, hidden_size, output_size)
    optimizer = torch.optim.Adam(model.parameters())

    train(model, train_data, val_data, 10)

    test_data = torch.load("test_data.pt")

    with torch.no_grad():
        correct = 0
        total = 0
        for input, target in test_data:
            output = model(input, target)
            _, predicted = output.max(1)
            correct += (predicted == target).sum().item()
            total += target.size(0)
        test_accuracy = correct / total
        print("Test accuracy: {:.4f}".format(test_accuracy))